In [10]:
# coding=utf-8
import xlrd
import requests
import re
import time
import os
import csv
import sys
import json
import pandas as pd
from config import headers,Cookie

initial_url='https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id_type=0'
next_url='https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id={}&max_id_type=0'
continue_url = initial_url
max_id_type=0
count=0
data_Dict = {'data':[]} 
fileName=''

def Get_CHN(text):
    dr = re.compile(r'<[^>]+>',re.S)
    dd = dr.sub('',text)
    data_Dict['data'].append(dd)
    write_to_excel(data_Dict)

def read_excel(path): #读取excel文档中的数据，返回一个DataFrame
    data = pd.read_excel(str(path),sheet_name=0)
    return pd.DataFrame(data)

def url_generator(_id,_mid): #生成初始网页URL和下一个网URL模板
    global initial_url,next_url
    initial_url='https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id_type=0' #一个循环后重置URL模板
    next_url='https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id={}&max_id_type={}'
    continue_url = initial_url
    
    initial_url = initial_url.format(str(_id),str(_mid))
    next_url=next_url.format(str(_id),str(_mid),"{}","{}")

def get_page(url): #请求页面
    try:
        requests.DEFAULT_RETRIES = 5  # 增加重试连接次数
        s = requests.session()
        s.keep_alive = False  # 关闭多余连接
        r = requests.get(url, headers=headers,cookies=Cookie)
        if r.status_code == 200:
            return r.json()
        else:
            if r.status_code == 200:
                return r.json()
    except requests.ConnectionError as e:
        print('error', e.args)

def get_data(data): #解析页面返回的json数据
    if data['ok']==1:
        global count
        if count>=15:
            max_id_type=1
        else:
            max_id_type=0
            
        max_id = data.get("data").get("max_id")
        if max_id != 0:
            comments = data.get('data').get('data')
            for arg in comments:
                Get_CHN(arg.get('text'))
            count+=1
            print(max_id)
            

            global next_url # 通过max_id生成下一个页面的url
            continue_url = next_url.format(str(max_id),max_id_type)
            print(continue_url)

            time.sleep(5)
            datafornextpage=get_page(continue_url) #访问下一个页面
            get_data(datafornextpage) #函数内递归调用
        else:
            comments = data.get('data').get('data')
            for arg in comments:
                Get_CHN(arg.get('text'))
    else:
        print('error')
        

def write_to_excel(content):
    pdData = pd.DataFrame(data_Dict)
    pdData.to_excel("{0}.xlsx".format(fileName),encoding='utf_8_sig')
    
if __name__ == "__main__":   
    excel_data = read_excel("#乐山五通桥市民因异味自发撤离#.xlsx")
    for count in range(0,excel_data['URL'].count()):
        global fileName
        fileName = str(excel_data['source'][count])
        url_generator(excel_data['id'][count],excel_data['mid'][count])
        jsondata = get_page(initial_url)
        get_data(jsondata)
        data_Dict = {'data':[]} 
        print("now to next account")

139258013983001
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=139258013983001&max_id_type=0
138158513817069
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=138158513817069&max_id_type=0
138158502432903
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=138158502432903&max_id_type=0
138158502359332
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=138158502359332&max_id_type=0
138158502121390
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=138158502121390&max_id_type=0
169811922181
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=169811922181&max_id_type=0
169793100981
https://m.weibo.cn/comments/hotflow?id=4540058705068663&mid=4540058705068663&max_id=169793100981&max_id_type=0
now to next account
